# Conexión con LLM

In [1]:
import os
import re
import json
import pandas as pd
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain.chat_models import ChatOpenAI
from langchain_voyageai import VoyageAIEmbeddings
from langchain_core.prompts import PromptTemplate

/Users/malenadiazrio/Documents/UOC/TFM/TFM_2025_1/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
load_dotenv(override=True)

True

In [8]:
RESOURCES_DIR="../../resources"
llm = ChatOpenAI(model="o4-mini")
embeddings_model = VoyageAIEmbeddings(model="voyage-3")

Definir la prompt

In [6]:
prompt = """Revisa cuidadosamente cada frase de la nota clínica para identificar términos relacionados con patrones de herencia genética, anomalías anatómicas, síntomas clínicos, hallazgos diagnósticos, resultados de pruebas y afecciones o síndromes específicos.
Ignora por completo los hallazgos negativos, los hallazgos normales (es decir, «normal» o «no»), los procedimientos y los antecedentes familiares. Incluye el contexto apropiado basándote únicamente en el pasaje.
Devuelve los términos extraídos en un objeto JSON con una única clave 'fenotipos', que contiene la lista de términos extraídos escritos correctamente. Asegúrate de que el resultado sea conciso, sin notas, comentarios ni metaexplicaciones adicionales. No dejes fuera adjetivos críticos para ese fenotipo.
Nota clínica: {clinical_note}"""

In [162]:
prompt = PromptTemplate.from_template(prompt)

Definir la entrada

In [7]:
test_df = pd.read_csv("../../datasets/TFM_test.csv")
test_df.annotations = test_df.annotations.apply(eval)
test_df.head(2)

,texts,annotations
0,Un hombre de 44 años con obesidad mórbida e ín...,"[HP:0001513, HP:0004394, HP:0005227, HP:001218..."
1,Un hombre de 32 años acudió a una unidad regio...,"[HP:0001903, HP:0003406, HP:0012185, HP:000439..."


In [ ]:
clinical_note = test_df.texts.iloc[0]
hpo_codes = test_df.annotations.iloc[0]
clinical_note

Primera llamada al LLM

In [163]:
chain = prompt | llm
answer = chain.invoke(
    {
        "clinical_note": clinical_note,
    }
)

In [164]:
def load_content(answer):
    clean_str = "".join(answer.content.split("{")[1:])
    clean_str = "".join(clean_str.split("}")[:-1])
    clean_str = '{' + clean_str + '}'
    return json.loads(clean_str)

In [165]:
fenotipos = load_content(answer)['fenotipos']

RAG

In [2]:
vectordb = Chroma(persist_directory="../../chroma_db/Voyage3", embedding_function=embeddings_model, 
                  collection_name="hpo_ontology_esp")

NameError: name 'Chroma' is not defined

In [168]:
candidatos = []
for fenotipo in fenotipos:
    candidatos.append(vectordb.similarity_search_with_score(fenotipo, k=10))

In [64]:
[document[0].id for document in candidatos[0]]

['HP:0045083', 'HP:0001513', 'HP:0031418', 'HP:0025502']

Segunda llamada al LLM

In [ ]:
prompt = """Identifica el término de la Ontología de Fenotipos Humanos (HPO) más apropiado para cada extracto de las notas clínicas del paciente a partir de una lista de candidatos (Código HPO - Descripción).
Da prioridad a los términos que sean concisos y directamente pertinentes para el síntoma o afección principal descritos. 
Céntrate en el tema central de cada frase y evita seleccionar opciones con detalles descriptivos o situacionales adicionales a menos que sean esenciales para captar con precisión el fenotipo. 
Asegúrate de que el término HPO elegido coincide estrechamente con la afección del paciente tal como se describe, sin añadir términos nuevos o extraños. 
Si hay varios candidatos, selecciona y devuelve el término HPO más pertinente que mejor represente la afección o síntoma primario. Proporciona sólo los códigos HPO elegidos en un json en formato lista con la clave "hpo_code", sin contexto ni comentarios adicionales."""

In [96]:
context = ""
for fenotipo, candidato in zip(fenotipos, candidatos):
    lista_candidatos = [f"{doc[0].id} - {doc[0].page_content}\n" for doc in candidato]
    context += f"""{fenotipo} 
Candidatos: {lista_candidatos}

"""

In [153]:
print(context)

obesidad mórbida 
Candidatos: ['HP:0045083 - obsoleto Aumento del índice de masa corporal.\n', 'HP:0001513 - Obesidad. Acumulación de un exceso considerable de grasa corporal.\n', 'HP:0031418 - Aumento del índice de masa corporal. Relación peso/altura al cuadrado anormalmente elevada, calculada dividiendo el peso del individuo en kilogramos por el cuadrado de la altura del individuo en metros y utilizada como indicador de sobrepeso en comparación con las medias.\n', 'HP:0025502 - Exceso de peso. Aumento del peso corporal con un índice de masa corporal de 25-29,9 kg por metro cuadrado.\n']

pólipos adenomatosos de la glándula fúndica 
Candidatos: ['HP:0033769 - Poliposis de las glándulas fúndicas. Múltiples pólipos en la mucosa secretora de ácido del cuerpo gástrico y el fundus. Los pólipos de la glándula fúndica (PGF) suelen tener un tamaño de 1 a 5 mm, aunque se han encontrado pólipos de mayor tamaño. Suelen ser sésiles, brillantes, translúcidos, de color pálido a rosado (parecido al 

In [99]:
prompt = f"""{prompt}
Fenotipos: 
{context}"""

In [102]:
response = llm.invoke(prompt)

In [110]:
codigos_hpo = load_content(response)["hpo_code"]
codigos_hpo

['HP:0001513',
 'HP:0033769',
 'HP:0004394',
 'HP:0033769',
 'HP:0100580',
 'HP:0200063',
 'HP:0005227',
 'HP:6001040',
 'HP:0012183',
 'HP:0031500']

In [126]:
print(f"""
Sobran: {set(codigos_hpo) - set(hpo_codes)}
Faltan: {set(hpo_codes) - set(codigos_hpo)}
""")


Sobran: {'HP:6001040', 'HP:0200063'}
Faltan: {'HP:0025501'}



In [136]:
with open(os.path.join(RESOURCES_DIR, "hpo_es.json"), "r") as fp:
    hpo = json.load(fp)

In [151]:
codigos_predichos = {doc.id:doc.page_content for doc in  vectordb.get_by_ids(set(codigos_hpo))}
codigos_reales = {doc.id:doc.page_content for doc in  vectordb.get_by_ids(set(hpo_codes))}

In [156]:
codigos_predichos['HP:0200063']

'Pólipos colorrectales. Múltiples crecimientos anormales que surgen del revestimiento del intestino grueso (colon o recto) y sobresalen en la luz intestinal.'

In [172]:
hpo_codigos_candidatos = []
for candidato in candidatos[0]:
    hpo_codigos_candidatos.append( candidato[0].id )

In [175]:
codigos_reales = {doc.id:doc.page_content for doc in  vectordb.get_by_ids(set(hpo_codigos_candidatos))}
codigos_reales

{'HP:0001513': 'Obesidad. Acumulación de un exceso considerable de grasa corporal.',
 'HP:0001956': 'Obesidad troncal. Obesidad localizada preferentemente en el tronco del cuerpo frente a las extremidades.',
 'HP:0004323': 'Anomalía del peso corporal. Aumento o disminución anormal del peso o distribución anormal de la masa corporal.',
 'HP:0004324': 'Aumento de peso corporal. Aumento anormal del peso corporal.',
 'HP:0012743': 'Obesidad abdominal. Exceso de grasa alrededor del estómago y el abdomen.',
 'HP:0025499': 'Obesidad clase I. Obesidad con un índice de masa corporal de 30 a 34,9 kg por metro cuadrado.',
 'HP:0025502': 'Exceso de peso. Aumento del peso corporal con un índice de masa corporal de 25-29,9 kg por metro cuadrado.',
 'HP:0025521': 'Aumento del porcentaje de grasa corporal. Porcentaje de grasa en relación con el peso corporal total superior a la norma, que suele definirse como el 32% para las mujeres y el 25% para los hombres.',
 'HP:0031418': 'Aumento del índice de ma

In [169]:
print(f"""
Sobran: {set(hpo_codigos_candidatos) - set(hpo_codes)}
Faltan: {set(hpo_codes) - set(hpo_codigos_candidatos)}
""")


Sobran: {'HP:0045083', 'HP:0010294', 'HP:0031493', 'HP:6001040', 'HP:0011459', 'HP:0020127', 'HP:0032609', 'HP:0004323', 'HP:0006753', 'HP:0011762', 'HP:0012743', 'HP:0004390', 'HP:0031857', 'HP:0004783', 'HP:0012028', 'HP:0034704', 'HP:0030255', 'HP:0008682', 'HP:0025521', 'HP:0002031', 'HP:0004324', 'HP:6000249', 'HP:0012737', 'HP:6000307', 'HP:0032952', 'HP:0002729', 'HP:0033140', 'HP:6000171', 'HP:0025502', 'HP:0100594', 'HP:0012718', 'HP:6001085', 'HP:0025270', 'HP:0034246', 'HP:0033314', 'HP:0005238', 'HP:0025499', 'HP:0004795', 'HP:0001956', 'HP:0004295', 'HP:0030393', 'HP:0032608', 'HP:0200008', 'HP:0012198', 'HP:0041050', 'HP:0031548', 'HP:0034945', 'HP:6000382', 'HP:0200063', 'HP:6001083', 'HP:0025758', 'HP:0030256', 'HP:0025761', 'HP:0032222', 'HP:0031418', 'HP:0034479', 'HP:0025198', 'HP:0100896'}
Faltan: {'HP:0025501'}



Raw OpenAI

In [9]:
prompt = """Eres una herramienta que sirve para extraer fenotipos de la ontología Human Phenotype Ontolgy a partir de notas clínicas para ello: 
1. Revisa cuidadosamente cada frase de la nota clínica para identificar términos relacionados con patrones de herencia genética, anomalías anatómicas, síntomas clínicos, hallazgos diagnósticos, resultados de pruebas y afecciones o síndromes específicos.
2. Ignora por completo los hallazgos negativos, los hallazgos normales (es decir, «normal» o «no»), los procedimientos y los antecedentes familiares. 
3. Para cada uno de los términos, piensa si tienen un fenotipo de la ontología HPO asociado. 
4. Para cada término, asigna el código HPO apropiado. 
Nota clínica: {clinical_note}"""
prompt = PromptTemplate.from_template(prompt)